## Python Mid term Part B Programming qs

## Name: Saranya Chintalapati
## Question 2

## Song database system



In [2]:
# import required packages
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra Cluster


In [4]:
#use session to talk to cassandra cluster
clstr = Cluster()
session = clstr.connect()

### Checking if there exists a keyspace called music 

For the requirement of this project, you will be creating a keyspace called "Music" in this cassandra-5 node system


In [22]:
session.execute("DROP KEYSPACE IF EXISTS music")

In [23]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(row[0])

newpy
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


### Create a new keyspace called music 

This keyspace is created with replaction factor of 3. Since it a songs database and one primary and 2 secondary nodes are sufficient to store the songs data. The class is chosen SimpleStrategy which specifies a simple replication factor for the cluster.

In [24]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor': 3
    }
""")

### Verify if the music keyspace is succesfully created using desc keyspaces

In [25]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(row[0])

music
newpy
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


### Create table songs_info to insert all the details required to store songs. 

The songs_info stores details about the song ID, which genre is it from, it duration and also the title of the song. The structure of the table is as follows:

* song_ID - Unique ID of the song
* title - title of the song
* genre - genre the song belongs to (pop, rock, classcial, country)
* duration - length of the song (in seconds)

Both title and song_ID are choosen as primary keys since it is easier to search for the songs using title names rather than song IDs. Therefore, title is also included in the primary key creation.


In [32]:
session.execute("""
CREATE TABLE IF NOT EXISTS music.songs_info ( 
    song_ID TEXT, 
    title TEXT, 
    duration FLOAT, 
    genre TEXT,
    PRIMARY KEY(title, song_ID)
);
""")

### Insert statement to insert songs data to songs_info table

In [33]:
# CQL INSERT statement
insert_query = """
INSERT INTO music.songs_info(song_ID, title, duration, genre)
VALUES (?, ?, ?, ?)
"""

prepared_insert = session.prepare(insert_query)

# Define  data to insert as a list of tuples
data_to_insert = [
    ('ID001', 'Prayers of the heaven', 180, 'pop'),
    ('ID002', 'Chery Chery', 201, 'rock'),
    ('ID003', 'Havanna', 236.5, 'classical'),
    ('ID004', 'Dont lie', 176, 'country'),
    ('ID005', 'Love me', 298, 'pop'),
    ('ID006', 'Halo', 278.5, 'rock')
]
# Insert each row in a loop
for row in data_to_insert:
    session.execute(prepared_insert, row)

### Query to select all the rows from the songs_info table 

In [34]:
rows = session.execute("select (song_ID, title, duration, genre) from music.songs_info")
for row in rows:
    print(f"song_ID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")

song_ID=ID004, title=Dont lie, duration=176.0, genre=country
song_ID=ID003, title=Havanna, duration=236.5, genre=classical
song_ID=ID005, title=Love me, duration=298.0, genre=pop
song_ID=ID006, title=Halo, duration=278.5, genre=rock
song_ID=ID002, title=Chery Chery, duration=201.0, genre=rock
song_ID=ID001, title=Prayers of the heaven, duration=180.0, genre=pop


### Update the genre of the song Halo to country. 

In [35]:
rows = session.execute("select (song_ID, title, duration, genre) from music.songs_info where title = 'Halo'")
for row in rows:
    print(f"song_ID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")

song_ID=ID006, title=Halo, duration=278.5, genre=rock


In [38]:
rows = session.execute("update music.songs_info set genre = 'country' where title = 'Halo' and song_ID = 'ID006'")

In [39]:
rows = session.execute("select (song_ID, title, duration, genre) from music.songs_info where title = 'Halo'")
for row in rows:
    print(f"song_ID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")

song_ID=ID006, title=Halo, duration=278.5, genre=country


### Delete statement to delete teh song with title Love me from songs_info table

In [40]:
rows = session.execute("delete from music.songs_info where song_ID = 'ID005' and title = 'Love me'")

In [41]:
rows = session.execute("select (song_ID, title, duration, genre) from music.songs_info where title = 'Love me'")
for row in rows:
    print(f"song_ID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")

In [50]:
rows = session.execute("select (song_ID, title, duration, genre) from music.songs_info")
for row in rows:
    print(f"song_ID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")

song_ID=ID004, title=Dont lie, duration=176.0, genre=country
song_ID=ID003, title=Havanna, duration=236.5, genre=classical
song_ID=ID006, title=Halo, duration=278.5, genre=country
song_ID=ID002, title=Chery Chery, duration=201.0, genre=rock
song_ID=ID001, title=Prayers of the heaven, duration=180.0, genre=pop


### Create an Index on genre to query songs_info table with genre filter rather than using title or song ID all the time

In [43]:
session.execute('CREATE INDEX ON music.songs_info(genre)')

In [52]:
rows = session.execute("select (song_ID, title, duration, genre) from music.songs_info where genre = 'classical' ")
for row in rows:
    print(f"song_ID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")

song_ID=ID003, title=Havanna, duration=236.5, genre=classical


### Range query to query the songs results with more than 200 seconds duration.

In [54]:
rows = session.execute("select (song_ID, title, duration, genre) from music.songs_info where duration > 200 ALLOW FILTERING")
for row in rows:
    print(f"song_ID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")

song_ID=ID003, title=Havanna, duration=236.5, genre=classical
song_ID=ID006, title=Halo, duration=278.5, genre=country
song_ID=ID002, title=Chery Chery, duration=201.0, genre=rock
